In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'C:\Users\feljo\OneDrive\Desktop\Dissertation programming\Feature Selection\Mutual Info\10 cls\mutualinfo_10cls.csv')
trainset = pd.read_csv(r'C:\Users\feljo\OneDrive\Desktop\Dissertation programming\Feature Selection\Mutual Info\10 cls\trainset_10cls.csv')
testset = pd.read_csv(r'C:\Users\feljo\OneDrive\Desktop\Dissertation programming\Feature Selection\Mutual Info\10 cls\testset_10cls.csv')

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from vecstack import stacking

In [4]:
#With LabelEncoder categorical values are changed into numeric values
#Trainset
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in trainset.columns:
        if trainset[column_name].dtype == object:
            trainset[column_name] = le.fit_transform(trainset[column_name])
        else:
            pass

In [5]:
#Testset
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in testset.columns:
        if testset[column_name].dtype == object:
            testset[column_name] = le.fit_transform(testset[column_name])
        else:
            pass

In [6]:
testset

BEDROOMS  DRYRFUEL  TOTCSQFT   KWHSPH   KWHCOL    KWHWTH   KWHRFG  \
0            2         5         0     0.00     0.00      0.00   346.47   
1            3         5      1952  1661.21  6129.16   4527.44   743.23   
2            1        -2         0     0.00     0.00      0.00  1093.25   
3            3         5       528   416.00   564.63      0.00  1206.75   
4            2         5         0     0.00     0.00      0.00  2401.29   
5            3         5         0   318.81     0.00      0.00   648.90   
6            2         5      1342     0.00   172.03    838.30   191.06   
7            3         5      3120     0.00  1069.48      0.00  1217.58   
8            3         5      1488  3800.32  6857.81   5279.34  1348.85   
9            4         5      1719     0.00  2343.34      0.00  2291.50   
10           3         1         0     0.00     0.00      0.00   448.58   
11           3         5      1420   576.37  3211.00      0.00  2369.57   
12           2        -2       115     0.00   156.79   1713.48   828.45   
13           4         5      2403  1644.28  5401.06   3020.53  1626.17   
14           4         5      2400     0.00  3552.26      0.00   760.97   
15           2         5         0     0.00     0.00      0.00   380.52   
16           3         5       558  1486.93   473.37  10513.41  2142.77   
17           2        -2       907  2143.18  2684.77   1363.13   689.55   
18           3         1      1155  1186.28   403.24      0.00  1258.65   
19           3         5      1640   498.30  4651.39   1003.46   864.23   
20           3         5       907   965.11  1292.40   1055.24   870.00   
21           3        -2         0     0.00     0.00   3429.59  1355.73   
22           2         1       773     0.00  1304.33      0.00   308.99   
23           4         5      2312     0.00  5744.83   2887.09  1578.13   
24           3         5      2174     0.00   494.65      0.00  1249.08   
25           4         5      2418     0.00   599.59      0.00  2645.93   
26           2         5       180     0.00  2495.12      0.00  2599.48   
27           3         5      1862  2652.08  4421.22   2321.62  1234.96   
28           3         5      1431  2342.19  1259.15   1730.14   578.73   
29           3         5         0     0.00     0.00      0.00  2922.66   
...        ...       ...       ...      ...      ...       ...      ...   
2991         3        -2       946   343.38  2495.16   1707.98  1097.84   
2992        -2        -2       866   924.31   535.00   1283.61   510.33   
2993         3         5      2790     0.00  8866.39   2778.15  2002.59   
2994         5         5      3412     3.77     0.74      1.99     0.91   
2995         3         1      1268     0.00  2634.06      0.00   705.29   
2996         4         5       480  4790.50  4579.05   6045.81  2088.79   
2997         1         5       640  2985.77   273.73      0.00   675.57   
2998         3         5      1307   913.05  4523.76   2700.17   620.38   
2999         4         1      2400     0.00   488.05      0.00  1339.29   
3000         3         1      1922   459.55   432.93      0.00   693.57   
3001         2         5         0  6048.52     0.00   5367.45  1055.86   
3002         2         5      2100   966.14  6188.30   1640.37   910.14   
3003         3         5         0   702.64     0.00      0.00  2477.18   
3004         2         1      2702     0.00  1439.23      0.00   953.35   
3005         4        -2       258     0.00   406.68      0.00  1070.36   
3006         5         5      1356     0.00  1202.63      0.00   740.34   
3007         3         5      1600  5306.07  2677.31      0.00  1942.68   
3008         2         5         0     0.00     0.00      0.00   715.28   
3009        -2         5       544  4003.29   905.69   2590.96  2368.37   
3010         4         5      2437     0.00   440.49      0.00  1833.81   
3011         2         1      1834     0.00  1353.52      0.00   761.18   
3012         2         5       

In [7]:
y_train = np.array(trainset['Class'])
y_train
x_train = (trainset.loc[:, trainset.columns != 'Class'])
x_train

BEDROOMS  DRYRFUEL  TOTCSQFT   KWHSPH    KWHCOL   KWHWTH   KWHRFG  \
0            3         5      1500   982.26   2452.57  1678.19   588.66   
1            3         5      1152  3728.19   1186.34  3864.64  1577.34   
2            3         1      1706     0.00    615.99     0.00  1727.89   
3            2         1      3786     0.00   1928.65     0.00  1442.03   
4            3         5      1434  1451.11   3089.80  1754.09   659.39   
5            2        -2       881     0.00    573.15     0.00   927.96   
6            4         5      1955   759.66    768.58     0.00  1147.16   
7            3         5       496  1927.95   1206.77     0.00  1013.76   
8            3         5       447     0.00    148.92  2666.70   512.57   
9            3         5      3003     0.00   7571.21     0.00  1941.27   
10           3         5      1649     0.00   4198.59     0.00  1006.60   
11           3         5      2852  1111.15   2328.77     0.00  2033.39   
12           3         1         0     0.00      0.00     0.00   833.31   
13           2        -2       423  3401.91    307.84     0.00   792.48   
14           1        -2         0  1109.55      0.00     0.00   167.78   
15           4         5      1157     0.00   1429.93     0.00  1399.17   
16           2        -2         0  1407.81      0.00     0.00   695.45   
17           3         5      1325  1314.50   1145.87     0.00   434.13   
18           3         5      2108  1426.65   5317.31  1514.24   689.86   
19           1        -2       702  1036.19    762.33  1693.11   575.67   
20           3         5      3060     0.00   2416.52     0.00  1198.93   
21           5         5      2636  2544.15   2665.45  2142.31  1055.17   
22           3         5      2077  2064.23   4577.74  4276.43  1074.18   
23           3         5      1280     0.00   1281.82     0.00  2424.91   
24           3         5      1705   771.23   5488.36  2557.63  1589.86   
25           4         5      1998  3568.38  15051.02  6202.55  2632.48   
26           5         5      5747     0.00   1101.38     0.00  2114.92   
27           3         5         0  1488.72      0.00  2012.72   853.98   
28           3         5      1994     0.00   1310.33     0.00  2762.62   
29           4         5      4464   272.81   7878.49     0.00  1481.42   
...        ...       ...       ...      ...       ...      ...      ...   
9032         1        -2         0     0.00      0.00     0.00   569.66   
9033         2         5       168     0.00      4.22  2397.14   814.59   
9034         3         5      1832   442.91   1503.51   685.76   830.16   
9035         3         5      1916   690.42   2331.33  3386.61  1171.70   
9036         3         1       800     0.00    348.37     0.00   608.70   
9037         3         1      2081     0.00   1177.77     0.00  1809.30   
9038         1         1         0   535.37      0.00     0.00    97.38   
9039         4         5      2377  2341.78   8222.48  3728.47  2174.38   
9040         4         1      3181     0.00   1327.98     0.00  2223.27   
9041         2         5      1180   336.23   1896.62     0.00  1163.15   
9042         3         5       439  2385.93   1048.44  1263.37   663.41   
9043         2         5       517     0.00     72.67     0.00   449.69   
9044         2        -2       297     0.00   1112.27  1603.82   819.39   
9045         3         5      1106  9482.35    591.49  4523.24  2335.19   
9046         5         2      8278     0.00   2906.83     0.00  5147.87   
9047         3         5      1960     0.00   3896.21     0.00  1173.46   
9048         3         5      1226     0.00    930.50     0.00   958.10   
9049         5         1      2216     0.00   3058.14     0.00  1586.49   
9050         3         5       586     0.00   1048.57     0.00   519.53   
9051         1         5       830  2504.95   1717.52  1823.07   961.59   
9052         3         5      1500  1119.54   3175.76  1796.26   740.19   
9053         2         5       

In [8]:
y_test = np.array(testset['Class'])
y_test
x_test = (testset.loc[:, testset.columns != 'Class'])
x_test

BEDROOMS  DRYRFUEL  TOTCSQFT   KWHSPH   KWHCOL    KWHWTH   KWHRFG  \
0            2         5         0     0.00     0.00      0.00   346.47   
1            3         5      1952  1661.21  6129.16   4527.44   743.23   
2            1        -2         0     0.00     0.00      0.00  1093.25   
3            3         5       528   416.00   564.63      0.00  1206.75   
4            2         5         0     0.00     0.00      0.00  2401.29   
5            3         5         0   318.81     0.00      0.00   648.90   
6            2         5      1342     0.00   172.03    838.30   191.06   
7            3         5      3120     0.00  1069.48      0.00  1217.58   
8            3         5      1488  3800.32  6857.81   5279.34  1348.85   
9            4         5      1719     0.00  2343.34      0.00  2291.50   
10           3         1         0     0.00     0.00      0.00   448.58   
11           3         5      1420   576.37  3211.00      0.00  2369.57   
12           2        -2       115     0.00   156.79   1713.48   828.45   
13           4         5      2403  1644.28  5401.06   3020.53  1626.17   
14           4         5      2400     0.00  3552.26      0.00   760.97   
15           2         5         0     0.00     0.00      0.00   380.52   
16           3         5       558  1486.93   473.37  10513.41  2142.77   
17           2        -2       907  2143.18  2684.77   1363.13   689.55   
18           3         1      1155  1186.28   403.24      0.00  1258.65   
19           3         5      1640   498.30  4651.39   1003.46   864.23   
20           3         5       907   965.11  1292.40   1055.24   870.00   
21           3        -2         0     0.00     0.00   3429.59  1355.73   
22           2         1       773     0.00  1304.33      0.00   308.99   
23           4         5      2312     0.00  5744.83   2887.09  1578.13   
24           3         5      2174     0.00   494.65      0.00  1249.08   
25           4         5      2418     0.00   599.59      0.00  2645.93   
26           2         5       180     0.00  2495.12      0.00  2599.48   
27           3         5      1862  2652.08  4421.22   2321.62  1234.96   
28           3         5      1431  2342.19  1259.15   1730.14   578.73   
29           3         5         0     0.00     0.00      0.00  2922.66   
...        ...       ...       ...      ...      ...       ...      ...   
2991         3        -2       946   343.38  2495.16   1707.98  1097.84   
2992        -2        -2       866   924.31   535.00   1283.61   510.33   
2993         3         5      2790     0.00  8866.39   2778.15  2002.59   
2994         5         5      3412     3.77     0.74      1.99     0.91   
2995         3         1      1268     0.00  2634.06      0.00   705.29   
2996         4         5       480  4790.50  4579.05   6045.81  2088.79   
2997         1         5       640  2985.77   273.73      0.00   675.57   
2998         3         5      1307   913.05  4523.76   2700.17   620.38   
2999         4         1      2400     0.00   488.05      0.00  1339.29   
3000         3         1      1922   459.55   432.93      0.00   693.57   
3001         2         5         0  6048.52     0.00   5367.45  1055.86   
3002         2         5      2100   966.14  6188.30   1640.37   910.14   
3003         3         5         0   702.64     0.00      0.00  2477.18   
3004         2         1      2702     0.00  1439.23      0.00   953.35   
3005         4        -2       258     0.00   406.68      0.00  1070.36   
3006         5         5      1356     0.00  1202.63      0.00   740.34   
3007         3         5      1600  5306.07  2677.31      0.00  1942.68   
3008         2         5         0     0.00     0.00      0.00   715.28   
3009        -2         5       544  4003.29   905.69   2590.96  2368.37   
3010         4         5      2437     0.00   440.49      0.00  1833.81   
3011         2         1      1834     0.00  1353.52      0.00   761.18   
3012         2         5       

In [9]:
models = [
    KNeighborsClassifier(n_neighbors=5),
    RandomForestClassifier(n_estimators= 10, random_state= 42),
    MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=500,activation = 'relu',solver='adam',random_state=1)
]

In [10]:
S_train, S_test = stacking(models,                   
                           x_train, y_train, x_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [10]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.56324372]
    fold  1:  [0.56569665]
    fold  2:  [0.56114790]
    fold  3:  [0.55176991]
    ----
    MEAN:     [0.56046455] + [0.00527168]
    FULL:     [0.56047230]

model  1:     [RandomForestClassifier]
    fold  0:  [0.78580873]
    fold  1:  [0.78835979]
    fold  2:  [0.78852097]
    fold  3:  [0.77389381]
    ----
    MEAN:     [0.78414582] + [0.00601599]
    FULL:     [0.78415361]

model  2:     [MLPClassifier]
    fold  0:  [0.11018070]
    fold  1:  [0.38051146]
    fold  2:  [0.19735099]
    fold  3:  [0.34646018]
    ----
    MEAN:     [0.25862583] + [0.10995629]
    FULL:     [0.25855220]



In [11]:
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.5, 
                      n_estimators=100, max_depth=2)

In [12]:
model = model.fit(S_train, y_train)
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.5, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [13]:
y_pred = model.predict(S_test)

In [14]:
print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))

Final prediction score: [0.81264482]


In [15]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))


[[255  21   0   0   0   0   1  23   0   0]
 [ 55 238  19   1   0   0   0   1   0   0]
 [  2  38 230   0   0   0   0   0   0  12]
 [  0   0   0 273  22   0   0   0  10   0]
 [  0   0   0  30 247  12   0   0   0   0]
 [  0   0   0   1  50 226  13   1   0   0]
 [  2   0   0   0   5  60 228  23   0   0]
 [ 46   4   0   0   0   6  40 205   0   0]
 [  0   0   0  32   0   0   0   0 266   0]
 [  0   0  36   0   0   0   0   0   0 287]]


In [17]:
df_confusion = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion

Predicted    0    1    2    3    4    5    6    7    8    9   All
Actual                                                           
0          255   21    0    0    0    0    1   23    0    0   300
1           55  238   19    1    0    0    0    1    0    0   314
2            2   38  230    0    0    0    0    0    0   12   282
3            0    0    0  273   22    0    0    0   10    0   305
4            0    0    0   30  247   12    0    0    0    0   289
5            0    0    0    1   50  226   13    1    0    0   291
6            2    0    0    0    5   60  228   23    0    0   318
7           46    4    0    0    0    6   40  205    0    0   301
8            0    0    0   32    0    0    0    0  266    0   298
9            0    0   36    0    0    0    0    0    0  287   323
All        360  301  285  337  324  304  282  253  276  299  3021